# Create baseline model
route and origin+destination locations

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import numpy as np
import pandas as pd

import shapely.ops as so
import shapely.geometry as sg
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as cx

import networkx as nx
import momepy

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

## Get network data

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

In [ ]:
# Get basic pedestrian network with widths & crossings
df = gpd.read_file(cf.output_file_crossings)
df['length'] = df['geometry'].length  # TODO remove once this is added in a previous notebook
df.tail(3)

## Get start and end location

In [ ]:
# Define origin and destination

# Test without crossings
#df_coords = pd.DataFrame({"coordinates": ["origin", "destination"],
#        "latitude": [52.35544169630071, 52.35646800424747],
#        "longitude": [4.7986989333355, 4.7979668425731345]})

# Mock-up data Nieuwmarkt/Lastage     
#df_coords = pd.DataFrame({"coordinates": ["origin", "destination"],
#        "latitude": [52.37078165480656, 52.36727669275347],
#        "longitude": [4.904291948879987, 4.90184804984204]})

# Mock-up data Osdorp-Midden
df_coords = pd.DataFrame({"coordinates": ["origin", "destination"],
        "latitude": [52.35775604247446, 52.36102160116833],
        "longitude": [4.793441885105594, 4.794255970874044]})

# Create geodataframe
gdf_coords = gpd.GeoDataFrame(
    df_coords, geometry=gpd.points_from_xy(df_coords.longitude, df_coords.latitude), crs="EPSG:4326"
)
gdf_coords = gdf_coords.to_crs(st.CRS)
gdf_coords = gdf_coords[['coordinates', 'geometry']]

## Create objective

In [ ]:
# User inputs
min_sidewalk_width = 1.4  # m
max_curb_height = 0.04  # m

In [ ]:
# Create accessible width parameter  # TODO deal with unknown and None, we now assume they are wide enough
df['accessible_width'] = 1
if min_sidewalk_width <= 0.9:
    df.loc[df['obstacle_free_width'].isin(['<0.9m']), 'accessible_width'] = 10**6
elif (min_sidewalk_width > 0.9) & (min_sidewalk_width <= 1.5):
    df.loc[df['obstacle_free_width'].isin(['<0.9m', '0.9-1.5m']), 'accessible_width'] = 10**6
elif (min_sidewalk_width > 1.5) & (min_sidewalk_width <= 2.0):
    df.loc[df['obstacle_free_width'].isin(['<0.9m', '0.9-1.5m', '1.5-2.0m']), 'accessible_width'] = 10**6
elif (min_sidewalk_width > 2.0) & (min_sidewalk_width <= 2.2):
    df.loc[df['obstacle_free_width'].isin(['<0.9m', '0.9-1.5m', '1.5-2.0m', '2.0-2.2m']), 'accessible_width'] = 10**6
elif (min_sidewalk_width > 2.2) & (min_sidewalk_width <= 2.9):
    df.loc[df['obstacle_free_width'].isin(['<0.9m', '0.9-1.5m', '1.5-2.0m', '2.0-2.2m', '2.2-2.9m']), 'accessible_width'] = 10**6
elif (min_sidewalk_width > 2.9) & (min_sidewalk_width <= 3.6):
    df.loc[df['obstacle_free_width'].isin(['<0.9m', '0.9-1.5m', '1.5-2.0m', '2.0-2.2m', '2.2-2.9m', '2.9-3.6m']), 'accessible_width'] = 10**6

In [ ]:
# Create accessible height parameter (TODO)
df['accessible_height'] = 1

In [ ]:
# Define weight (combination of objectives) -- DUMMY version for now
df['my_weight'] = df['length']*df['accessible_width']*df['accessible_height']
#df.to_csv('test_routes.csv', sep=";")
df.head()
#df['my_weight'].plot()

In [ ]:
G = momepy.gdf_to_nx(df, approach="primal", multigraph=True)

## Get route

In [ ]:
# Get origin and destination location
origin_point = gdf_coords.loc[gdf_coords['coordinates'] == 'origin', 'geometry'].values[0]
dest_point = gdf_coords.loc[gdf_coords['coordinates'] == 'destination', 'geometry'].values[0]

# Get origin and destination node location
origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

# Get origin and destination node
origin_node = (origin_node_loc.x, origin_node_loc.y)
dest_node = (dest_node_loc.x, dest_node_loc.y)   

In [ ]:
# Get 'shortest' path between origin and destination, based on custom weight
my_path_weight = nx.shortest_path(G, origin_node, dest_node, weight='my_weight')

# Get 'shortest' path between origin and destination, based on length (for comparison)
my_path_length = nx.shortest_path(G, origin_node, dest_node) #, weight='length')

In [ ]:
# Put 'shortest' path in a dataframe, based on accessibility weight
G_path_weight = nx.subgraph(G, my_path_weight)
df_path_weight = momepy.nx_to_gdf(G_path_weight, lines=True, points=False)
df_path_weight.head(3)

In [ ]:
# Put 'shortest' path in a dataframe, based on length
G_path_length = nx.subgraph(G, my_path_length)
df_path_length = momepy.nx_to_gdf(G_path_length, lines=True, points=False)
df_path_length.head(3)

## Visualize

In [ ]:
# Subset network for plotting
my_rad = 70
gdf_coords['buffer'] = gdf_coords['geometry'].buffer(my_rad, cap_style=3)
plot_area = gpd.GeoDataFrame(geometry=[gdf_coords['buffer'][0].union(gdf_coords['buffer'][1])], crs=st.CRS)
df_sub = gpd.sjoin(df, plot_area, how='inner').reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# Network
df_sub.plot(ax=ax, color='lightgrey', linewidth=1)

# 'Shortest' paths
df_path_weight.plot(ax=ax, color='black', linewidth=3)
#df_path_length.plot(ax=ax, color='grey', linewidth=2)

# Origin and destination location
gdf_coords.head(1).plot(ax=ax, color='blue', markersize=50)
gdf_coords.tail(1).plot(ax=ax, color='red', markersize=50)

# Origin and destination nodes
gpd.GeoSeries([origin_node_loc], crs=st.CRS).plot(ax=ax, color='blue', markersize=20)
gpd.GeoSeries([dest_node_loc], crs=st.CRS).plot(ax=ax, color='red', markersize=20)

# Background
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Voyager, crs=st.CRS)

# Legend
route = mpatches.Patch(color='grey', label='shortest route')
route_acc = mpatches.Patch(color='black', label='accessible route')
origin = mpatches.Patch(color='blue', label= 'origin')
dest = mpatches.Patch(color='red', label= 'destination')
#plt.legend(handles=[route,route_acc,origin,dest], loc='lower center')
plt.legend(handles=[route_acc,origin,dest], loc='lower center')

plt.axis('off')

#plt.savefig('../data/example_route_NL.png', bbox_inches='tight')
plt.savefig('../data/example_route_OM.png', bbox_inches='tight')

plt.show()

## Store data

In [ ]:
df_path_weight['length'] = df_path_weight['length'].round(2)
df_path_weight_store = df_path_weight.to_crs('epsg:4326')
df_path_weight_store = df_path_weight_store[['geometry', 'length']]
df_path_weight_store

In [ ]:
gdf_coords_store = gdf_coords.to_crs('epsg:4326')
gdf_coords_store = gdf_coords_store[['coordinates', 'geometry']]
gdf_coords_store

In [ ]:
#df_path_weight_store.to_csv('../data/example_route_NL.csv', sep=';')
df_path_weight_store.to_csv('../data/example_route_OM.csv', sep=';')
#df_path_weight_store.to_file('../data/example_route.gpkg', driver='GPKG')

#gdf_coords.to_csv('../data/example_locations_NL.csv', sep=";")
df_path_weight_store.to_csv('../data/example_locations_OM.csv', sep=';')
#gdf_coords.to_file('../data/example_locations.gpkg', driver='GPKG')